In [2]:
import pandas as pd
import numpy as np
import requests
import json
import re
import psycopg2
from sqlalchemy import create_engine

# Full insert by using .to_sql

In [6]:
engine = create_engine('postgresql+psycopg2://**.rds.amazonaws.com:5432/**')
conn = engine.connect()

price_data.to_sql(name='comp_price', con=engine, if_exists = 'replace', index=False)
conn.close()

# .to_sql insert by chunks

In [15]:
engine = create_engine('postgresql+psycopg2://**.rds.amazonaws.com:5432/**')
chunks = pd.read_csv('adj_prices.csv', chunksize=1000)

for i, chunk in enumerate(chunks):
    print(i)
    chunk['shcode'] = chunk['code'].str[1:]
    chunk.drop('code', axis=1, inplace=True)
    chunk.to_sql(name='comp_price', con=engine, if_exists = 'append', index=False)

# Raw level bulk insert

- Much Much Faster than the .to_sql function

In [59]:
conn_string = "host='**' dbname ='**' user='**' password='**'"

try:
    conn = psycopg2.connect(conn_string)
except:
    print("error database connection")

curs = conn.cursor()


chunks = pd.read_csv('adj_prices.csv', chunksize=100000)
for i, df in enumerate(chunks):
    df['dt'] = pd.to_datetime(df['dt'])
    df['shcode'] = df['code'].str[1:]
    sql_query = '''
        INSERT INTO comp_price (date, shcode, open, high, low, close, volume)
            SELECT unnest( %(dates)s),
                   unnest( %(shcodes)s),
                   unnest( %(opens)s),
                   unnest( %(highs)s),
                   unnest( %(lows)s),
                   unnest( %(closes)s),
                   unnest( %(volumes)s)
        '''
    
    dataset = {}
    dataset['dates'] = df['dt'].tolist()
    dataset['shcodes'] = df['shcode'].tolist()
    dataset['opens'] = df['adj_open'].tolist()
    dataset['highs'] = df['adj_high'].tolist()
    dataset['lows'] = df['adj_low'].tolist()
    dataset['closes'] = df['adj_close'].tolist()
    dataset['volumes'] = df['adj_volume'].tolist()
    print(i, ' load data completed')

    curs.execute(sql_query, dataset)
    conn.commit()
conn.close()

0  load data completed
1  load data completed
2  load data completed
3  load data completed
4  load data completed
5  load data completed
6  load data completed
7  load data completed
8  load data completed
9  load data completed
10  load data completed
11  load data completed
12  load data completed
13  load data completed
14  load data completed
15  load data completed
16  load data completed
17  load data completed
18  load data completed
19  load data completed
20  load data completed
21  load data completed
22  load data completed
23  load data completed
24  load data completed
25  load data completed
26  load data completed
27  load data completed
28  load data completed
29  load data completed
30  load data completed
31  load data completed
